In [23]:
from forch import nn
import numpy as np
import forch
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(2, 10)
        self.fc2 = nn.Linear(10, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

    def backward(self):
        return super().backward()

In [19]:
model = Model()

In [20]:
model(np.array([[2, 3]]))

array([[0.14982278]])

In [ ]:
criton = nn.MSELoss()
optim = None